In [58]:
import os
import openai
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os
from tqdm import tqdm

In [59]:
OPENAI_TOKEN = os.getenv('OPENAI_TOKEN')

In [60]:
OPENAI_TOKEN
openai.api_key = OPENAI_TOKEN

In [61]:
file_path = 'data/raw/query_test.txt'
query_test  = []

with open(file_path, 'r') as file:
        # Read each line and append to the list
        for line in file:
            query_test.append(line.strip()) 

In [62]:
query_test

['Mammals that reside in the deepest parts of the ocean.',
 'Mammals that navigate using echolocation.',
 'Mammals that possess the ability to glide between trees.',
 'Mammals that exhibit hibernation during winter months.',
 'Mammals that are primarily nocturnal hunters.',
 'Mammals that form complex social structures within their groups.',
 'Mammals that burrow underground for shelter.',
 'Mammals that inhabit the densest rainforests.',
 'Mammals that have adapted to extreme cold climates.',
 'Mammals that communicate through a sophisticated system of calls.',
 'Mammals that use tools for various tasks.',
 'Mammals that have a keen sense of smell for foraging.',
 'Mammals that possess fur adapted for camouflage in snow.',
 'Mammals that can leap great distances.',
 'Mammals that rely on communal nesting.',
 'Mammals that possess a prehensile tail for gripping objects.',
 'Mammals that live in burrows dug into the ground.',
 'Mammals that are known for their migratory patterns.',
 'Ma

In [63]:
RAW_DATA_PATH = os.path.join("data", "raw")
PROCESSED_DATA_PATH = os.path.join("data", "processed")

In [64]:
data = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "data.csv"))
model = SentenceTransformer('model/')
index = faiss.read_index('mammals.index')

In [65]:
# define function to return top_k mammals dictionary based query

def fetch_mammals(dataframe_idx):
    info = data.iloc[dataframe_idx]
    meta = dict()
    meta['organism_name'] = info['organism_name']
    return meta


def search(query, top_k, index, model):
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    result_id = top_k[1].tolist()[0]
    result_id = list(np.unique(result_id))
    results = [fetch_mammals(idx) for idx in result_id]
    return results



In [66]:
top_k = 3
test = len(query_test)
amount = top_k * test
right = 0


for query in tqdm(query_test):
    results = search(query, top_k=top_k, index=index, model=model)
    for result in results:
        text = result['organism_name']
        response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[
            {"role": "system", "content": 'You are an animal mammals expert.'},
            {"role": "user", "content": f'"{text}"\nwhether this organism fall/include within the given description "{query}". answer with "Yes" or "No" only'}
        ], max_tokens=1000)

        answer = response["choices"][0]["message"]["content"]
        isMammals = 'yes' in answer.lower()
        
        if isMammals:
            right += 1

100%|██████████| 50/50 [01:28<00:00,  1.77s/it]


In [67]:
right/amount * 100

70.0